In [1]:
#Sktime imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.shapelet_based import MrSEQLClassifier
from sklearn.model_selection import GroupShuffleSplit

from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
import sys
from os.path import exists

import importlib

# EEG package modules__________________________________________________________________
import constants
import custom_ts_length
# import eeg_stat_ts
import format_eeg_data
import run_expts

In [2]:
import warnings

In [3]:
warnings.warn('Warning Message: 4')

/tmp/ipykernel_702237/2806487782.py:1: UserWarning: Warning Message: 4
  warnings.warn('Warning Message: 4')


In [4]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [5]:
from sktime.registry import all_estimators
all_estimators()

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 10002. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sktime/registry/_lookup.py:156: ImportWarning: No module named 'esig'. 'esig' is a soft dependency and not included in the sktime installation. Please run: `pip install esig`. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(str(e), ImportWarning)
/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/lib/python3.9/site-packages/sktime/registry/_lookup.py:156: ImportWarning: No module named 'pmdarima

[('AggrDist', sktime.dists_kernels.compose_tab_to_panel.AggrDist),
 ('Arsenal', sktime.classification.kernel_based._arsenal.Arsenal),
 ('AutoCorrelationTransformer',
  sktime.transformations.series.acf.AutoCorrelationTransformer),
 ('AutoETS', sktime.forecasting.ets.AutoETS),
 ('AutoEnsembleForecaster',
  sktime.forecasting.compose._ensemble.AutoEnsembleForecaster),
 ('BOSSEnsemble', sktime.classification.dictionary_based._boss.BOSSEnsemble),
 ('BoxCoxTransformer', sktime.transformations.series.boxcox.BoxCoxTransformer),
 ('CanonicalIntervalForest',
  sktime.classification.interval_based._cif.CanonicalIntervalForest),
 ('Catch22', sktime.transformations.panel.catch22.Catch22),
 ('Catch22ForestClassifier',
  sktime.classification.hybrid._catch22_forest_classifier.Catch22ForestClassifier),
 ('ClaSPSegmentation', sktime.annotation.clasp.ClaSPSegmentation),
 ('ClaSPTransformer', sktime.transformations.series.clasp.ClaSPTransformer),
 ('ColumnConcatenator',
  sktime.transformations.panel.co

In [6]:
from sklearn.dummy import DummyClassifier

In [7]:
from sktime.classification.distance_based import ProximityTree

In [8]:
from sktime.classification.dictionary_based import (
    MUSE,
    WEASEL,
    BOSSEnsemble,
    ContractableBOSS,
    TemporalDictionaryEnsemble
)

In [9]:
import sys
print(sys.executable)

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_only_with_eeg_pckg/bin/python


In [10]:
'''
o 1. PD vs HC,   2.(PD , PD+RBD) vs HC,   3.(PD + PD&RBD) vs HC vs RBD  multiclass, 4.PD vs PD+RBD vs RBD vs HC 
'''
core_path = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/'

#1 - This is to run the custom length expts
folders = {'N1': 'N1_customised_40_expt_files/' , 'N2' : 'N2_120_expt_files_v2/','N3' : 'N3_82_expt_files/', 'REM' :'REM_19_expt_files/'}
#2 -This is to run the expts on the full original length data without any dropping or cutting of the TS 
folders = {'N1': 'N1_full_expts/' , 'N2' : 'N2_full_expts/','N3' : 'N3_full_expts/', 'REM' :'REM_full_expts/'}
#3 - This is to run the wakeful eyes closed experiments
folders = {'EC1': 'EC_8_expts/' , 'EC2' : 'EC_full_expts/' }

#Folder = folders[sys.argv[1]] #User can input which experiments will be run
print('Change the folder here ^_^! ')

folder = folders['EC1'] #I'm choosing this because it's the TS & Therefore quickest hopefully?
folder = 'REM_full_expts/'

expts = ['MrSQL_expt_1', 'MrSQL_expt_2' , 'MrSQL_expt_3' , 'MrSQL_expt_4']

power_band_cols = ['Delta', 'Theta', 'Alpha', 'Sigma', 'Beta', 'Gamma', 'TotalAbsPow']

regions = ['Prefrontal','Frontal', 'Left Frontal', 'Right Frontal', 'Central', 'Left Central', 
           'Right Central', 'Left Temporal','Right Temporal', 'Parietal', 'Left Parietal', 'Right Parietal', 'Occipital']

#Load the appropriate X (full set of features ) , y (full set of labels) , groups 
load_path = core_path + folder
X = pd.read_hdf(load_path + 'X.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'y_full.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'groups.h5', key='df', mode='r')  

Change the folder here ^_^! 


In [11]:
load_path

'/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/REM_full_expts/'

In [12]:
subsets_dict_new = {}
for col in power_band_cols:
    name = col + '_' + 'Prefrontal'
    subsets_dict_new[name] = [name]

# Order 1 
subsets_dict_3_bpw =  {'Delta_Prefrontal': ['Delta_Prefrontal'], 'Theta_Prefrontal': ['Theta_Prefrontal'], 'Alpha_Prefrontal': ['Alpha_Prefrontal'],}

#Order 2
# subsets_dict_3_bpw =  {'Alpha_Prefrontal': ['Alpha_Prefrontal'],'Theta_Prefrontal': ['Theta_Prefrontal'], 'Delta_Prefrontal': ['Delta_Prefrontal']}


# subsets_dict_new = {'Gamma_Prefrontal': ['Gamma_Prefrontal']}
# subsets_dict_new = {'Delta_Prefrontal': ['Delta_Prefrontal']}
subsets_dict_Theta = {'Theta_Prefrontal': ['Theta_Prefrontal']}
subsets_dict_Delta = {'Delta_Prefrontal': ['Delta_Prefrontal']}

subsets_dict_new = subsets_dict_Delta

In [13]:
# Edited this notebook to have three types of dictionary in order 

In [14]:
clfs = {'MrSQL_1' : MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) }
# clfs = {'WEASEL' : WEASEL(binning_strategy="equi-depth", anova=False, random_state=47) }
# clfs = {'ProxTree' : ProximityTree(random_state=5) }
# clfs =  { 'Dummy' : DummyClassifier() }
# clfs = {'KNN' : KNeighborsTimeSeriesClassifier() }

dicts = [subsets_dict_Theta,  subsets_dict_Delta , subsets_dict_3_bpw]
dicts = [subsets_dict_3_bpw , subsets_dict_Theta,  subsets_dict_Delta]


results_dfs = []
for subsets_dict_new in dicts:
    
    results_dict = {}
    # for num in [1,2,3,4]:
    for num in [1]:
        print('commencing for ' + str(num) + '...')
        
        clfs = {'MrSQL_1' : MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) }


        #1.Generate expt specific X,y,groups
        X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups,num)

        t1 = time.time()
        results_df = run_expts.run_mv_tsc(X_expt ,y_expt , groups_expt, clfs , return_df = True,subset_names_and_cols = subsets_dict_new, random_states = [13 , 37])

        t2 = time.time()

        results_dict[num] = results_df
        
        acc_std_results_df = run_expts.generate_subset_acc_std(results_dict[num], return_df = True)
        results_dfs.append(acc_std_results_df)

        print(t2-t1)

for df in results_dfs:
    display(df)

commencing for 1...
You are using a test version of this run_clf_with_CV function


/export/sphere/ebirah/ko20929/eeg_package/src/run_expts.py:163: UserWarning: run_clf_with_CV : Test version of function , classifier is always MrSEQL Classifier
  warnings.warn('run_clf_with_CV : Test version of function , classifier is always MrSEQL Classifier')


You are using a test version of this run_clf_with_CV function


/export/sphere/ebirah/ko20929/eeg_package/src/run_expts.py:163: UserWarning: run_clf_with_CV : Test version of function , classifier is always MrSEQL Classifier
  warnings.warn('run_clf_with_CV : Test version of function , classifier is always MrSEQL Classifier')


You are using a test version of this run_clf_with_CV function


...

You are using a test version of this run_clf_with_CV function
22.65544319152832


/export/sphere/ebirah/ko20929/eeg_package/src/run_expts.py:163: UserWarning: run_clf_with_CV : Test version of function , classifier is always MrSEQL Classifier
  warnings.warn('run_clf_with_CV : Test version of function , classifier is always MrSEQL Classifier')


,random_state,Delta_Prefrontal_MrSQL_1_acc,Delta_Prefrontal_MrSQL_1_std,Theta_Prefrontal_MrSQL_1_acc,Theta_Prefrontal_MrSQL_1_std,Alpha_Prefrontal_MrSQL_1_acc,Alpha_Prefrontal_MrSQL_1_std
0,13,0.663968,0.196470,0.527857,0.144108,0.597302,0.162447
1,37,0.637222,0.154196,0.617222,0.151213,0.615000,0.139660


,random_state,Theta_Prefrontal_MrSQL_1_acc,Theta_Prefrontal_MrSQL_1_std
0,13,0.527857,0.144108
1,37,0.617222,0.151213


,random_state,Delta_Prefrontal_MrSQL_1_acc,Delta_Prefrontal_MrSQL_1_std
0,13,0.663968,0.196470
1,37,0.637222,0.154196
